# Open Trust Fabric (OTF)
# Digital Platform Use Case
# 04- Mobility Index Analysis
June 2021

# Description

This notbook provides a preliminary analysis of potential links and correlation between digital platform contracts and urban mobility.

The urban mobility index is a metric indicating the total mobility in a given city compared to its reference value over time. The mobility index therefore indicates whether overall mobility has increased or decreased in a city over time.

The mobility index has been downloaded from this website: https://citymapper.com/

This metric can be particularly useful while assessing the overall movements of the population during pandemic times. 

In [1]:
import pandas as pd
import numpy as np
import os
import gzip

In [2]:
df_mob = pd.read_csv('Citymapper_Mobility_Index_20210609.csv', skiprows=3)

In [12]:
df_mob.head(2)

,Date,Amsterdam,Barcelona,Birmingham,Boston,Brussels,Chicago,Copenhagen,Hong Kong,Istanbul,...,Seoul,Singapore,St. Petersburg,Stockholm,Sydney,São Paulo,Toronto,Vancouver,Vienna,Washington DC
0,2020-01-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.04,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df_mob['Date'] = pd.to_datetime(df_mob['Date'])

In [14]:
df_mob.set_index('Date', inplace=True)

In [15]:
df_mob.columns = [col.lower() for col in list(df_mob)]

In [16]:
df_mob.shape

(506, 36)

In [18]:
df_mob.head(2)

,amsterdam,barcelona,birmingham,boston,brussels,chicago,copenhagen,hong kong,istanbul,lisbon,...,seoul,singapore,st. petersburg,stockholm,sydney,são paulo,toronto,vancouver,vienna,washington dc
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.07,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.04,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The table above shows a considerable number of missing values in the observations of the urban mobility index. The high presence of missing observations might limit the potential of the urban mobility analysis.

In [4]:
data_path = '../../../Data/UC1/data_cleaned_storical/'

with gzip.open(data_path + 'POLICY_NETWORK.csv (1).gzip') as file_in:
    df = pd.read_csv(file_in, low_memory=False)

In [5]:
df.head()

,MODEL_INSTANCE,TIME_STAMP,PARTIES_PROPOSER_ID,PARTIES_PARTICIPANT_ID,ASSET_ID,ASSET_PRICE,ASSET_MIGRATION,ASSET_LOCATION,ASSETS_REVIEW_SCORES_ACCURACY,ASSET_NUMBER_REVIEWS,ASSET_AVAILABILITY_30,YEAR,MONTH
0,PJNs4Fpyg4,2015-04-10,62142,30537860,15883,NaN,NaN,NaN,NaN,NaN,NaN,2015,4
1,rGvohKWuaC,2016-06-19,62142,37529754,15883,$85.00,0.0,vienna,10.0,1.0,9.0,2016,6
2,O1ZE3GDKxL,2016-07-29,62142,3147341,15883,$85.00,0.0,vienna,10.0,2.0,17.0,2016,7
3,2t2vxoLsH5,2016-08-13,62142,29518067,15883,$85.00,0.0,vienna,9.0,3.0,9.0,2016,8
4,KBHJiEPuFf,2016-11-21,62142,36016357,15883,$85.00,0.0,vienna,10.0,4.0,9.0,2016,11


In [19]:
df.rename(columns={'TIME_STAMP' : 'Date'}, inplace=True)

In [20]:
df_gb = pd.pivot_table(data = df,
                       index=["Date"],
                       values=["PARTIES_PROPOSER_ID", "ASSET_ID", "MODEL_INSTANCE"],
                       columns = "ASSET_LOCATION",
                       aggfunc={"MODEL_INSTANCE" : "count"})

In [23]:
df_gb.columns = df_gb.columns.droplevel(0)

In [24]:
df_gb.isnull().sum()/len(df_gb) * 100

ASSET_LOCATION
amsterdam     33.448674
athens        66.493656
brussels      63.148789
copenhagen    65.051903
dublin        75.490196
lisbon        70.126874
madrid        59.573241
prague        67.589389
rome          83.737024
stockholm     96.482122
vienna        28.258362
dtype: float64

In [25]:
df_gb.head()

ASSET_LOCATION,amsterdam,athens,brussels,copenhagen,dublin,lisbon,madrid,prague,rome,stockholm,vienna
Date,,,,,,,,,,,
2015-07-01,NaN,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.0
2015-07-02,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.0
2015-07-03,NaN,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0
2015-07-04,NaN,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.0
2015-07-05,NaN,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.0


In [26]:
df_mob = df_mob.loc[:, ['amsterdam', 'vienna']]

In [27]:
df_mob.rename(columns={c : c + '_mob' for c in list(df_mob)}, inplace=True)

In [33]:
df_merged = df_gb.loc[:, ['amsterdam', 'vienna']].merge(df_mob, left_index=True, right_index=True, how='inner')

In [34]:
df_merged.isnull().sum()/ len(df_merged) * 100

amsterdam        52.577320
vienna           15.979381
amsterdam_mob    21.649485
vienna_mob       21.649485
dtype: float64

In [38]:
df_merged.dropna(how = 'any').head(10)

,amsterdam,vienna,amsterdam_mob,vienna_mob
Date,,,,
2020-05-01,3.0,19.0,0.07,0.09
2020-05-02,3.0,7.0,0.06,0.13
2020-05-03,9.0,14.0,0.08,0.08
2020-05-04,5.0,3.0,0.10,0.16
2020-05-05,10.0,7.0,0.09,0.14
2020-05-06,5.0,3.0,0.09,0.17
2020-05-07,3.0,8.0,0.08,0.17
2020-05-08,6.0,12.0,0.09,0.15
2020-05-09,6.0,5.0,0.09,0.12


The table above shows the days in which both digital platform and urban mobility index were jointly available. The high presence of missing values limits the possibility to perform in-depth analysis of urban mobility and the number of contracts exchanged in the digital platfom.

In [35]:
df_merged.corr()

,amsterdam,vienna,amsterdam_mob,vienna_mob
amsterdam,1.000000,0.950169,0.755948,0.601523
vienna,0.950169,1.000000,0.841034,0.766225
amsterdam_mob,0.755948,0.841034,1.000000,0.961214
vienna_mob,0.601523,0.766225,0.961214,1.000000


The figure above shows initial evidence of a positive correlation across the number of contracts and urban mobility index. The correlation represented above cannot be considered as conclusive as it is based on a small number of observations (10), however, it might represent the possibility to compute a correlation between urban mobility and the number of contracts exchanged within a digital platform. 